In [1]:
!source /etc/profile

import os
import sys
import random
import time
from collections import deque
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn import preprocessing
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization, Activation, Flatten, MaxPooling2D, Conv2D
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from tensorflow.keras import optimizers
from tensorflow.keras import utils
import keras_tuner as kt
import joblib

# CD to top level git directory
if ".git" not in os.listdir("."):
    os.chdir("../../")

# Check GPU
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
# tf.config.list_physical_devices()

Num GPUs Available:  1


2022-06-06 03:05:27.373399: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-06 03:05:27.455809: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-06 03:05:27.456491: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


In [2]:
import gzip
compressed_bstr = b"\x1f\x8b\x08\x00\xdf\xd7\x7e\x62\x02\xff\xa5\x55\x4b\x8f\xdb\x20\x10\x3e\x67\x7f\x05\xe5\x04\x92\x13\xed\xa3\x2f\x45\xbb\xab\x5e\xaa\xaa\x97\x6a\x2f\x3d\x59\x16\x22\x06\xa7\x53\x61\xec\x00\xde\xcd\xb6\xea\x7f\xef\x80\x13\x39\x91\x9d\x55\xba\xbd\xd8\x30\x7c\xf3\xcd\x83\x99\x01\xea\xb6\x71\x81\xf8\x6e\xd5\xba\xa6\xd4\xde\x13\xe9\x89\x6f\x2f\x4a\x23\x71\xfd\xe5\xe1\xfb\xf2\x62\xa6\x74\x45\x84\x00\x0b\x41\x08\xe6\xb5\xa9\x38\x0a\x67\x71\xb1\x10\x42\xc9\x20\x85\x20\x77\xe4\x5b\x63\xf5\x1e\xbb\xd6\xa1\x97\x0f\x70\xa7\x43\xe7\x2c\x52\x2f\x5c\x67\x19\xa3\xf6\x11\x14\xc8\xb9\xaf\x81\x66\x84\xce\x37\xe9\xbb\xa5\x3c\x23\x3e\xa8\xa6\x0b\x77\x88\x7c\xf8\xfa\xf0\x99\x2f\xfa\xfd\x42\xe9\xb2\x51\x9a\xd1\x2e\x54\xf3\x8f\x14\xc5\xad\x81\x60\xc0\x6a\xcf\xf8\xe0\x63\x14\x08\x0c\xa5\xd5\x2e\x3c\xef\xec\x67\x44\x85\xe7\x56\x67\xa4\xb5\xb2\xd6\xc9\x1b\xa8\xc8\x91\xff\x51\x36\x8b\x4b\x0c\xe4\xe8\x00\xe5\xda\x78\x3d\x71\x3e\xc4\xc8\x87\xf0\x72\x96\x4c\x31\xe8\xdd\x63\xf4\x9e\xf2\xfc\xaa\xd8\xef\x6e\x71\x77\xb9\xdf\xc5\x25\xcf\xce\x43\xce\xaf\x0a\x4e\xaa\xc6\x11\x20\x60\x49\x72\x04\x43\x48\xf1\x44\x01\x14\x08\xbe\x98\x7d\xda\x07\xde\xe7\x23\x9e\x8e\x2e\x20\x7f\xd1\xe0\x60\x63\x1c\x65\xb4\x48\x6f\xd1\x84\xea\xca\x20\x22\xf9\x3d\x3d\x6d\xbc\xd6\xf5\x60\xfb\xa5\xcc\x85\x26\x48\x93\x91\xce\x6b\x95\x91\xca\x69\x8d\xc0\x9c\x81\x0d\x2c\x82\x72\x28\xce\x4e\xe5\xbf\xe0\x8f\xf2\xe9\xa4\x5d\x6b\x66\xb4\x4d\x26\xf9\x2e\xd2\x6a\x25\x30\x88\xc6\x3d\x8b\xce\xcb\xf5\x2e\xd8\xde\x06\x5e\x07\x41\xe5\x97\x31\xd7\x67\x60\x6e\x8a\xa2\xbf\x1a\x0c\x3a\x15\x0e\xc5\x3e\x60\xcd\xea\xa7\x2e\x43\x86\xbd\xf0\x9b\xa6\x74\x41\xd0\xb5\x10\x74\x49\x8c\xac\x57\x4a\x12\x9f\xc1\x12\x93\x34\x4a\x1d\xcf\x73\x9a\x84\xb1\x9b\xa2\x38\xfe\xe3\x01\x2d\x16\x60\x95\xde\xb2\x0d\x2f\x62\x74\xe0\xc1\xfa\x20\x6d\xa9\xd9\x26\x76\x9c\xe3\x24\x96\x39\x99\xa2\xdc\xf4\x55\xb1\x89\x6e\x63\x7a\xe3\x6d\x23\xab\x10\x4e\xb7\xee\xc8\xa9\xe5\x94\xfa\x62\x0f\x64\xfc\x0f\x3f\x6c\x15\xfc\x8d\x8a\x26\x25\x68\x3c\x33\xfa\xb2\x19\x37\x37\x16\x09\x7a\x72\xbb\x6e\x3b\xd1\x05\x30\x78\xe1\x23\x46\x4c\x5c\xfb\x2a\xc2\xa8\x38\x45\xd8\x36\x4f\xda\x9d\xcf\x58\x99\x46\x26\xce\xa4\x27\x94\x93\x4f\x53\xac\xb5\xdc\xfe\x1f\xb1\x81\x1a\xc2\x14\x73\xd9\x29\x29\x1e\xb5\xf3\xd0\xd8\xf3\xd9\xb1\x22\x22\xf7\xa1\xf2\x7d\x6a\xa0\x11\xbf\x72\x80\x80\x57\x5b\x38\x56\x3f\x61\x43\x1a\x73\xfa\xdd\x99\x1c\x2a\x32\x04\xe7\xe3\x20\xd9\xa6\xda\xdd\xa6\x96\x83\x5d\x7a\x63\x03\xd8\x26\x90\x2d\x3e\x2d\xd2\x05\xff\x04\xe1\x07\xc3\x82\xa6\x9c\x48\xab\x10\xfc\xe6\x8e\x50\xb4\x49\x8b\x43\xa6\xb4\xc8\xe3\x5c\x02\x7e\x30\x27\xe9\xdb\x9b\x0f\xef\xdf\x5d\x5f\x5e\xd1\xa1\x93\x15\x94\x81\xfd\x82\x96\x25\x9d\x8c\xe4\x83\x8f\x51\x02\xab\x2e\x68\x74\x54\xf6\x23\x48\x46\x9a\x84\x2c\x38\x3f\xf9\xb0\x1e\xb6\x0d\xd6\x27\xca\xf1\x71\x66\xfc\x2f\x17\xd9\x71\x54\xc0\x07\x00\x00"
exec(gzip.decompress(compressed_bstr))

gpu.name

'Tesla T4'

In [12]:
# Data parameters
SEQ_LEN = 336 #hours
FUTURE_PERIOD_PREDICT = 1 #hours

# Hyperparameter Optimizer parameters
MAX_TRIALS = 100 # 10t x 20e ~ 4h

# Model parameters
EPOCHS = 50
BATCH_SIZE = 16

NAME = "TRANSFORMER-01"

In [4]:
## Import Data

train_df = pd.read_csv("data/formatted/BTCUSDT-1h-data.csv")
train_df.set_index("timestamp", inplace=True)
train_df

,open,high,low,close,volume,rsi,ema
timestamp,,,,,,,
1503064800,4304.15,4371.52,4296.04,4356.31,51.563675,52.623958,4327.151560
1503068400,4356.31,4357.37,4302.72,4340.31,24.093449,51.678528,4327.804777
1503072000,4320.52,4340.31,4287.79,4331.71,15.118957,51.167386,4327.995329
1503075600,4302.97,4318.16,4221.05,4293.09,46.533767,48.919621,4326.319858
1503079200,4293.09,4293.09,4193.70,4259.40,74.368943,47.054235,4323.157459
...,...,...,...,...,...,...,...
1642464000,42174.66,42318.61,42095.69,42238.31,858.935040,40.857704,42832.082979
1642467600,42238.31,42306.96,42013.12,42083.47,698.707920,38.983032,42807.934173
1642471200,42078.64,42148.21,41609.78,41724.20,1463.109620,35.115683,42772.975006


In [5]:
## Add min max bounds to data

price_max = 80000.0
volume_max = 60000.0
rsi_max = 100.0

price_min = 2000.0
volume_min = 0.0
rsi_min = 0.0

max_df = pd.DataFrame()

max_df["timestamp"] = []

for col in train_df.columns:
    max_df[col] = []

max_df = max_df.append({"timestamp": str(int(time.time())),
                "open": price_max,
                "high": price_max,
                "low": price_max,
                "close": price_max,
                "volume": volume_max,
                "rsi": rsi_max,
                "ema": price_max,
                "target": price_max}, ignore_index=True)

max_df = max_df.append({"timestamp": str(int(time.time())),
                "open": price_min,
                "high": price_min,
                "low": price_min,
                "close": price_min,
                "volume": volume_min,
                "rsi": rsi_min,
                "ema": price_min,
                "target": price_min}, ignore_index=True)

max_df.set_index("timestamp", inplace=True)

max_df.head()

,open,high,low,close,volume,rsi,ema,target
timestamp,,,,,,,,
1654484729,80000.0,80000.0,80000.0,80000.0,60000.0,100.0,80000.0,80000.0
1654484729,2000.0,2000.0,2000.0,2000.0,0.0,0.0,2000.0,2000.0


In [6]:
## Formatting data and Scaler Initialization

# def classify(current, future):
#     return float((future - current) / current)


train_df["target"] = train_df["close"].shift(-FUTURE_PERIOD_PREDICT)

# # Cut off NaNs
# # data = data[:-FUTURE_PERIOD_PREDICT]
train_df.dropna(inplace=True)

# data["target"] = list(map(classify, data["close"], data["future"]))
# # data[["close", "future", "target"]].tail()
# data = data.drop("future", 1)

# Fit scalers
price_scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))
volume_scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))
rsi_scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))

train_df = train_df.append(max_df)

price_scaler.fit(np.array(train_df["close"]).reshape(-1, 1))
volume_scaler.fit(np.array(train_df["volume"]).reshape(-1, 1))
rsi_scaler.fit(np.array(train_df["rsi"]).reshape(-1, 1))

#Dump scalers

try:
    os.mkdir(f"scalers/{NAME}")
except:
    pass

joblib.dump(price_scaler, f"scalers/{NAME}/price_scaler")
joblib.dump(volume_scaler, f"scalers/{NAME}/volume_scaler")
joblib.dump(rsi_scaler, f"scalers/{NAME}/rsi_scaler")

# Remove min max boundary values
train_df = train_df[:-2]


# Split dataset # Dont split here; do split after shuffle
# last_5_pct = int(len(data) * .95)

# train_data = data[:last_5_pct]
# validation_data = data[last_5_pct:]

# print(f"{len(train_data)} :: {len(validation_data)}")

train_df.head()

,open,high,low,close,volume,rsi,ema,target
timestamp,,,,,,,,
1503064800,4304.15,4371.52,4296.04,4356.31,51.563675,52.623958,4327.151560,4340.31
1503068400,4356.31,4357.37,4302.72,4340.31,24.093449,51.678528,4327.804777,4331.71
1503072000,4320.52,4340.31,4287.79,4331.71,15.118957,51.167386,4327.995329,4293.09
1503075600,4302.97,4318.16,4221.05,4293.09,46.533767,48.919621,4326.319858,4259.40
1503079200,4293.09,4293.09,4193.70,4259.40,74.368943,47.054235,4323.157459,4236.89


In [7]:
## Preprocess Data

def preprocess_df_p1(df_p):

    df = pd.DataFrame()
    for col in df_p.columns:
        df[col] = df_p[col]

    for col in df.columns:
        scaler = None
        if col in ["open", "high", "low", "close", "ema", "target"]:
            scaler = price_scaler
        elif col == "volume":
            scaler = volume_scaler
        elif col == "rsi":
            scaler = rsi_scaler
        else:
            raise Exception("Column not recognized and scaler cannot be determined")

        df.replace([np.inf, -np.inf], np.nan, inplace=True)
        df.dropna(inplace=True)
        df[col] = scaler.transform(np.array(df[col]).reshape(-1, 1))

        
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)

    sequential_data = []
    prev_periods = deque(maxlen=SEQ_LEN)

    for i in df.values:
        prev_periods.append([n for n in i[:-1]])
        if len(prev_periods) == SEQ_LEN:
            sequential_data.append([np.array(prev_periods), i[-1]])

    return sequential_data
    # random.shuffle(sequential_data)

def preprocess_df_p2(seq_data):

    # Balance buys and sells
    buys = []
    sells = []

    for seq, target in seq_data:

        if target < seq[-1][3]: #compares to close column
            sells.append([seq, target])
        elif target > seq[-1][3]:
            buys.append([seq, target])

    lower = min(len(buys), len(sells))

    buys = buys[:lower]
    sells = sells[:lower]

    local_seq_data = buys + sells

    random.shuffle(local_seq_data)

    X = [d[0] for d in local_seq_data]
    Y = [d[1] for d in local_seq_data]

    return np.array(X), np.array(Y)
    

# train_x, train_y = preprocess_df(train_data)
# validation_x, validation_y = preprocess_df(validation_data)

# Preprocess and split data here
seq_data_full = preprocess_df_p1(train_df)

last_5_pct = int(len(seq_data_full) * .95)

seq_data_train = seq_data_full[:last_5_pct]
seq_data_val = seq_data_full[last_5_pct:]

train_x, train_y = preprocess_df_p2(seq_data_train)
validation_x, validation_y = preprocess_df_p2(seq_data_val)

In [8]:
joblib.dump(train_x, "data/pickle/train_x.dump")
joblib.dump(train_y, "data/pickle/train_y.dump")
joblib.dump(validation_x, "data/pickle/validation_x.dump")
joblib.dump(validation_y, "data/pickle/validation_y.dump")

['data/pickle/validation_y.dump']

## Build the model

Our model processes a tensor of shape `(batch size, sequence length, features)`,
where `sequence length` is the number of time steps and `features` is each input
timeseries.

You can replace your classification RNN layers with this one: the
inputs are fully compatible!

We include residual connections, layer normalization, and dropout.
The resulting layer can be stacked multiple times.

The projection layers are implemented through `keras.layers.Conv1D`.

In [8]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res


The main part of our model is now complete. We can stack multiple of those
`transformer_encoder` blocks and we can also proceed to add the final
Multi-Layer Perceptron classification head. Apart from a stack of `Dense`
layers, we need to reduce the output tensor of the `TransformerEncoder` part of
our model down to a vector of features for each data point in the current
batch. A common way to achieve this is to use a pooling layer. For
this example, a `GlobalAveragePooling1D` layer is sufficient.

In [9]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(1, activation="linear")(x)
    return keras.Model(inputs, outputs)


In [10]:
def build_model_hp(hp):
    kwargs = {
        "input_shape": train_x.shape[1:],
        "head_size": hp.Int("head_size", min_value=32, max_value=1024, step=32),
        "num_heads": hp.Int("num_heads", min_value=2, max_value=10, step=1),
        "ff_dim": 4,
        "num_transformer_blocks": hp.Int("num_transformer_blocks", min_value=2, max_value=10, step=1),
        "mlp_units": [hp.Int(f"mlp_units_{i}", min_value=32, max_value=512, step=32) for i in range(hp.Int("mlp_count", min_value=1, max_value=4, step=1))],
        "mlp_dropout": hp.Float("mlp_dropout", min_value=0.1, max_value=0.7, step=0.1),
        "dropout": hp.Float("dropout", min_value=0.1, max_value=0.5, step=0.1),
    }

    model = build_model(**kwargs)

    model.compile(
        loss="mape",
        optimizer=keras.optimizers.Adam(learning_rate=hp.Choice("learning_rate", values=[.001, .0001, .00001, .000001])),
        metrics=["mse"],
    )

    return model

In [ ]:
# input_shape = train_x.shape[1:]

# model = build_model(
#     input_shape,
#     head_size=256,
#     num_heads=4,
#     ff_dim=4,
#     num_transformer_blocks=4,
#     mlp_units=[128],
#     mlp_dropout=0.4,
#     dropout=0.25,
# )

# model.compile(
#     loss="mse",
#     optimizer=keras.optimizers.Adam(learning_rate=1e-4),
#     metrics=["mape"],
# )
# model.summary()

# callbacks = [keras.callbacks.EarlyStopping(patience=50, restore_best_weights=True)]



In [11]:
# Initialize Tuner

tuner = kt.BayesianOptimization(
                    max_trials=MAX_TRIALS,
                    hypermodel=build_model_hp,
                    objective="val_loss",
                    overwrite=False,
                    directory=f"tuners/{NAME}",
                    project_name=f"{NAME}",)

tuner.search_space_summary(extended=True)

callbacks = [
                keras.callbacks.EarlyStopping(patience=50, restore_best_weights=True),
                keras.callbacks.TensorBoard(log_dir=f"logs/{NAME}",
                                                 histogram_freq=1,
                                                 profile_batch='500,520')

            ]

2022-06-06 03:05:47.567912: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-06 03:05:47.569016: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-06 03:05:47.570020: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-06 03:05:47.570907: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

Search space summary
Default search space size: 8
head_size (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 1024, 'step': 32, 'sampling': None}
num_heads (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 10, 'step': 1, 'sampling': None}
num_transformer_blocks (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 10, 'step': 1, 'sampling': None}
mlp_count (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 4, 'step': 1, 'sampling': None}
mlp_units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
mlp_dropout (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.7, 'step': 0.1, 'sampling': None}
dropout (Float)
{'default': 0.1, 'conditions': [], 'min_value': 0.1, 'max_value': 0.5, 'step': 0.1, 'sampling': None}
learning_rate (Choice)
{'default': 0.001, 'conditions': [], 'values': [0.001, 0.0001, 1e-05, 1e-06], 'ordered': True}


2022-06-06 03:05:50.329916: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-06-06 03:05:50.329950: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-06-06 03:05:50.334179: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1630] Profiler found 1 GPUs
2022-06-06 03:05:50.334755: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcupti.so.11.2'; dlerror: libcupti.so.11.2: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/targets/x86_64-linux/lib:/opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/lib:/usr/lib:/usr/local/cuda/lib:/usr/local/cuda/lib64:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/targets/x86_64-linux/lib:/opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/lib:/usr/lib:
2022-06-06 03:05:50.606242: I tensorflow/cor

In [13]:
with tf.device("/device:GPU:0"):
    tuner.search(train_x, train_y, 
                epochs=EPOCHS,
                batch_size=BATCH_SIZE,
                callbacks=callbacks,
                validation_data=(validation_x, validation_y))


Search: Running Trial #1

Hyperparameter    |Value             |Best Value So Far 
head_size         |352               |?                 
num_heads         |4                 |?                 
num_transformer...|5                 |?                 
mlp_count         |2                 |?                 
mlp_units_0       |64                |?                 
mlp_dropout       |0.6               |?                 
dropout           |0.2               |?                 
learning_rate     |0.001             |?                 

Epoch 1/50


2022-06-06 03:06:48.873785: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201
2022-06-06 03:06:50.704930: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 11.0.221, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


 499/2212 [=====>........................] - ETA: 2:51 - loss: 154.9052 - mse: 0.7478

2022-06-06 03:07:41.753293: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-06-06 03:07:41.753333: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.


 519/2212 [======>.......................] - ETA: 2:51 - loss: 152.9581 - mse: 0.7390

2022-06-06 03:07:44.433954: I tensorflow/core/profiler/lib/profiler_session.cc:67] Profiler session collecting data.
2022-06-06 03:07:44.454882: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed
2022-06-06 03:07:44.732646: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:526]  GpuTracer has collected 23668 callback api events and 23605 activity events. 
2022-06-06 03:07:45.181678: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-06-06 03:07:45.832927: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/TRANSFORMER-01/ebb627fea4f12d9329bcf086bd6c9046/execution0/plugins/profile/2022_06_06_03_07_45

2022-06-06 03:07:46.232664: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/TRANSFORMER-01/ebb627fea4f12d9329bcf086bd6c9046/execution0/plugins/profile/2022_06_06_03_07_45/ip-172-31-0-83.trace.json.gz


 520/2212 [======>.......................] - ETA: 2:59 - loss: 152.8236 - mse: 0.7384

2022-06-06 03:07:46.759971: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/TRANSFORMER-01/ebb627fea4f12d9329bcf086bd6c9046/execution0/plugins/profile/2022_06_06_03_07_45

2022-06-06 03:07:46.772392: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to logs/TRANSFORMER-01/ebb627fea4f12d9329bcf086bd6c9046/execution0/plugins/profile/2022_06_06_03_07_45/ip-172-31-0-83.memory_profile.json.gz
2022-06-06 03:07:46.783549: I tensorflow/core/profiler/rpc/client/capture_profile.cc:251] Creating directory: logs/TRANSFORMER-01/ebb627fea4f12d9329bcf086bd6c9046/execution0/plugins/profile/2022_06_06_03_07_45
Dumped tool data for xplane.pb to logs/TRANSFORMER-01/ebb627fea4f12d9329bcf086bd6c9046/execution0/plugins/profile/2022_06_06_03_07_45/ip-172-31-0-83.xplane.pb
Dumped tool data for overview_page.pb to logs/TRANSFORMER-01/ebb627fea4f12d9329bcf086bd6c9046/execution0/plugins/profile/2022_06_06_03_07_45/ip

2212/2212 [==============================] - 241s 104ms/step - loss: 113.1006 - mse: 0.5720 - val_loss: 140.4782 - val_mse: 0.1538
Epoch 2/50
1412/2212 [==================>...........] - ETA: 1:23 - loss: 98.3269 - mse: 0.5143

KeyboardInterrupt: 